In [15]:
!dvc run --force -n gen_env -w .. -d data/default_env.pickle -p true_norm,colour_specific,shape_specific,target_area python -o data/env.pickle scripts/gen_env.py


Stage 'gen_env' is cached - skipping run, checking out outputs        core>
Modifying stage 'gen_env' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock
